In [24]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import plotly.express as px

from datetime import datetime
from scipy import stats

#Set Display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [25]:
#Pick Season
season = "2020-21"

#Import Files
all_players = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_players.csv', parse_dates= ["news_added"])
all_teams = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_teams.csv')
all_events = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_events.csv', parse_dates= ["deadline_time"])
player_types = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_types.csv')
game_phases = pd.read_csv(r'./data/' + season + '/Cleaned_data/game_phases.csv')

player_season_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_season_history.csv')
player_gameweek_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_gameweek_history.csv')
#player_gameweek_history = pd.read_csv(r'./data/2019-20/player_past_history/player_gameweek_history_raw.csv')
player_future_fixture = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_future_fixture.csv', parse_dates = ["kickoff_time"])



In [26]:
player_future_fixture.head()

,fixture_id,fixture_code,home_team_id,team_h_score,away_team_id,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty,player_id,opponent_team_id,team_id
0,23,2128310,11,NaN,1,NaN,3.0,False,0,False,2020-09-28 19:00:00,Gameweek 3,False,5,1,11,1
1,29,2128316,1,NaN,15,NaN,4.0,False,0,False,2020-10-04 13:00:00,Gameweek 4,True,3,1,15,1
2,44,2128331,12,NaN,1,NaN,5.0,False,0,False,2020-10-17 14:00:00,Gameweek 5,False,5,1,12,1
3,49,2128336,1,NaN,9,NaN,6.0,False,0,False,2020-10-24 14:00:00,Gameweek 6,True,3,1,9,1
4,64,2128351,13,NaN,1,NaN,7.0,False,0,False,2020-10-31 15:00:00,Gameweek 7,False,4,1,13,1


In [27]:
def player_future(player_id):
    data = player_future_fixture[player_future_fixture.player_id == player_id]

    player = all_players[all_players.player_id == player_id]
    team = data.team_id.iloc[0]
    goals_scored = player.iloc[0].goals_scored
    player_name = player.iloc[0]['web_name']

    fig = px.line(
        data, 
        x = 'event_name', 
        y = 'difficulty', 
        title = player_name,
        hover_data=['is_home'])

    fig.show()

In [28]:
player_future(150)

In [29]:
    opponent_strength = player_future_fixture['difficulty']
    opponent_strength
    #opponent_strength = player_analysis.iloc[:,2]

0        5
1        3
2        5
3        3
4        4
        ..
19690    3
19691    4
19692    4
19693    2
19694    2
Name: difficulty, Length: 19695, dtype: int64

In [30]:
def get_SMA(player_id , count):
    data = player_future_fixture[player_future_fixture.player_id == player_id]
    opponent_strength = data['difficulty']
    # List to store moving average results
    SMA2 = list(range(0, len(opponent_strength) - count))

    # Compute moving average
    for ii in range(len(SMA2)):
        # Previous gw index
        index = range(ii, ii + count)
        # Get data for previous gw
        prev_gw2 = opponent_strength.iloc[index]

        # Sum previous gw
        summation2 = np.sum(prev_gw2)

        # Get average
        avg2 = summation2/count
        # Save results to list
        SMA2[ii] = avg2
         
    # Define column label
    label2 = f"Opponent strength SMA"

    
    # Get corresponding gw for moving_avg
    gameweeks2 = opponent_strength.index[count:]
    gameweek = data.event_name[count:]
    
    # Convert list into Pandas Series
    SMA2 = pd.Series(SMA2, name = label2, index = gameweek)
    
    
    return SMA2

In [55]:
def player_future_avg(player_id , no_games):
    data = get_SMA(player_id , no_games)

    player = all_players[all_players.player_id == player_id]

    player_name = player.iloc[0]['web_name']
    title = "Average difficulty next " + str(no_games) + " games for " + str(player_name)

    fig = px.line(
        data, 
        title = title)

    fig.show()

In [59]:
player_future_avg(200 , 4)

In [58]:
result = get_SMA(150,4)
result

event_name
Gameweek 7     2.50
Gameweek 8     2.75
Gameweek 9     3.25
Gameweek 10    2.75
Gameweek 11    2.75
Gameweek 12    2.75
Gameweek 13    2.50
Gameweek 14    3.00
Gameweek 15    3.25
Gameweek 16    3.50
Gameweek 17    3.75
Gameweek 18    3.25
Gameweek 19    3.25
Gameweek 20    2.75
Gameweek 21    2.50
Gameweek 22    2.50
Gameweek 23    2.25
Gameweek 24    2.75
Gameweek 25    2.50
Gameweek 26    3.25
Gameweek 27    3.50
Gameweek 28    3.50
Gameweek 29    3.75
Gameweek 30    3.00
Gameweek 31    2.75
Gameweek 32    2.25
Gameweek 33    2.25
Gameweek 34    2.75
Gameweek 35    2.75
Gameweek 36    2.75
Gameweek 37    2.75
Gameweek 38    2.50
Name: Opponent strength SMA, dtype: float64

## FANTASY PLAYER VALUE

In [63]:
all_players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round',
       'player_code', 'cost_change_event', 'cost_change_event_fall',
       'cost_change_start', 'cost_change_start_fall', 'dreamteam_count',
       'player_type_id', 'ep_next', 'ep_this', 'event_points', 'form',
       'player_id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo',
       'PPG', 'selected_by_percent', 'status', 'team_id', 'team_code',
       'total_points', 'transfers_in_event', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank', 'threat_rank_type',
       'ict_index_rank', 'ict_index_rank_type',
       'corners_and

In [65]:
top_players = all_players[['web_name' , 'now_cost', 'status' , 'player_type_id', 'total_points' , 'minutes' , 'PP90' , 'PPG' , 'PPMM' , 'VAPM']][all_players['minutes']>100][all_players['status'] != "i"].sort_values(by='VAPM' , ascending = False).head(60)
top_players.head(20)

,web_name,now_cost,status,player_type_id,total_points,minutes,PP90,PPG,PPMM,VAPM
245,Castagne,56,a,2,23,180,11.500000,11.5,2.053571,1.696429
257,Bamford,57,a,4,21,130,14.538462,10.5,1.842105,1.491228
183,Calvert-Lewin,72,a,4,25,158,14.240506,12.5,1.736111,1.458333
260,Costa,56,a,3,19,180,9.500000,9.5,1.696429,1.339286
156,Zaha,71,a,3,23,180,11.500000,11.5,1.619718,1.338028
243,Justin,46,a,2,16,180,8.000000,8.0,1.739130,1.304348
27,Gabriel,51,a,2,17,180,8.500000,8.5,1.666667,1.274510
253,Klich,56,a,3,18,170,9.529412,9.0,1.607143,1.250000
446,Son,90,a,3,26,180,13.000000,13.0,1.444444,1.222222
384,Hendrick,50,a,3,16,180,8.000000,8.0,1.600000,1.200000


In [66]:
top_players[top_players['player_type_id'] == 1].head(3)

,web_name,now_cost,status,player_type_id,total_points,minutes,PP90,PPG,PPMM,VAPM
286,Alisson,60,a,1,15,180,7.5,7.5,1.25,0.916667
143,Guaita,50,a,1,13,180,6.5,6.5,1.30,0.900000
7,Leno,50,a,1,9,180,4.5,4.5,0.90,0.500000


In [67]:
top_players[top_players['player_type_id'] == 2].head(5)

,web_name,now_cost,status,player_type_id,total_points,minutes,PP90,PPG,PPMM,VAPM
245,Castagne,56,a,2,23,180,11.5,11.5,2.053571,1.696429
243,Justin,46,a,2,16,180,8.0,8.0,1.739130,1.304348
27,Gabriel,51,a,2,17,180,8.5,8.5,1.666667,1.274510
526,Saïss,51,a,2,16,180,8.0,8.0,1.568627,1.176471
134,James,51,a,2,15,180,7.5,7.5,1.470588,1.078431


In [68]:
top_players[top_players['player_type_id'] == 3].head(5)

,web_name,now_cost,status,player_type_id,total_points,minutes,PP90,PPG,PPMM,VAPM
260,Costa,56,a,3,19,180,9.500000,9.5,1.696429,1.339286
156,Zaha,71,a,3,23,180,11.500000,11.5,1.619718,1.338028
253,Klich,56,a,3,18,170,9.529412,9.0,1.607143,1.250000
446,Son,90,a,3,26,180,13.000000,13.0,1.444444,1.222222
384,Hendrick,50,a,3,16,180,8.000000,8.0,1.600000,1.200000


In [69]:
top_players[top_players['player_type_id'] == 4].head(3)

,web_name,now_cost,status,player_type_id,total_points,minutes,PP90,PPG,PPMM,VAPM
257,Bamford,57,a,4,21,130,14.538462,10.5,1.842105,1.491228
183,Calvert-Lewin,72,a,4,25,158,14.240506,12.5,1.736111,1.458333
74,Maupay,65,a,4,18,180,9.000000,9.0,1.384615,1.076923


In [71]:
price_gk = top_players['now_cost'][top_players['player_type_id'] == 1].head(2).sum()
price_def = top_players['now_cost'][top_players['player_type_id'] == 2].head(5).sum()
price_mid = top_players['now_cost'][top_players['player_type_id'] == 3].head(5).sum()
price_fwd = top_players['now_cost'][top_players['player_type_id'] == 4].head(3).sum()


team_cost = price_gk + price_def + price_mid + price_fwd
print("Price for the best team is " + str(np.divide(team_cost,10)))

Price for the best team is 88.2


In [87]:
def player_value():

    fig = px.scatter(
        top_players.head(30), 
        x = 'now_cost', 
        y = 'VAPM',
        #size = 'minutes',
        color = 'player_type_id',
        text = 'web_name',
        hover_data=['PPG'])

    fig.show()

In [88]:
player_value()